### Marketing Analytics Case Study 1

In [1]:
import psycopg2 as pg2
import hidden
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio

In [2]:
# Load the secrets
secrets = hidden.secrets()

In [3]:
#establish the connection
conn = pg2.connect(host= secrets['host'], 
                  port = secrets['port'],
                  database=secrets['database'],
                  user=secrets['user'],
                  password=secrets['pass'],
                  connect_timeout=3)

In [4]:
#retrieve the cursor
cur = conn.cursor()

In [5]:
#execute the query
sql_query = """
DROP TABLE IF EXISTS complete_joint_dataset;
CREATE TEMP TABLE complete_joint_dataset AS
SELECT
  rental.customer_id,
  inventory.film_id,
  film.title,
  rental.rental_date,
  category.name AS category_name
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
  ON rental.inventory_id = inventory.inventory_id
INNER JOIN dvd_rentals.film
  ON inventory.film_id = film.film_id
INNER JOIN dvd_rentals.film_category
  ON film.film_id = film_category.film_id
INNER JOIN dvd_rentals.category
  ON film_category.category_id = category.category_id;

SELECT * FROM complete_joint_dataset;
"""
cur.execute(sql_query)

In [6]:
cur.fetchone() #returns the first row of the data

(130,
 80,
 'BLANKET BEVERLY',
 datetime.datetime(2005, 5, 24, 22, 53, 30),
 'Family')

In [123]:
customer_data = sqlio.read_sql_query(sql_query,conn)

In [124]:
customer_data

,customer_id,film_id,title,rental_date,category_name
0,130,80,BLANKET BEVERLY,2005-05-24 22:53:30,Family
1,459,333,FREAKY POCUS,2005-05-24 22:54:33,Music
2,408,373,GRADUATE LORD,2005-05-24 23:03:39,Children
3,333,535,LOVE SUICIDES,2005-05-24 23:04:41,Horror
4,222,450,IDOLS SNATCHERS,2005-05-24 23:05:21,Children
...,...,...,...,...,...
16039,14,168,COMANCHEROS ENEMY,2005-08-23 22:25:26,Children
16040,74,951,VOYAGE LEGALLY,2005-08-23 22:26:47,Classics
16041,114,452,ILLUSION AMELIE,2005-08-23 22:42:48,Foreign
16042,103,439,HUNCHBACK IMPOSSIBLE,2005-08-23 22:43:07,Drama


### Tasks: 
* category_name: The name of the top 2 ranking categories
* rental_count: How many total films have they watched in this category
* average_comparison: How many more films has the customer watched compared to the average DVD Rental Co customer?
* percentile: How does the customer rank in terms of the top X% compared to all other customers in this film category?
* category_percentage: What proportion of total films watched does this category make up?

In [125]:
customer_data.head()

,customer_id,film_id,title,rental_date,category_name
0,130,80,BLANKET BEVERLY,2005-05-24 22:53:30,Family
1,459,333,FREAKY POCUS,2005-05-24 22:54:33,Music
2,408,373,GRADUATE LORD,2005-05-24 23:03:39,Children
3,333,535,LOVE SUICIDES,2005-05-24 23:04:41,Horror
4,222,450,IDOLS SNATCHERS,2005-05-24 23:05:21,Children


In [260]:
query1= customer_data.copy()

In [261]:
#rental_count: How many total films have they watched in this category
query1 = query1.groupby(['customer_id','category_name'])
category_rental_counts = query1.size().to_frame(name='rental_count')

In [262]:
rank_category = category_rental_counts.groupby('customer_id')['rental_count'].rank(method= 'dense',ascending=False).to_frame(name='rank_category')

In [263]:
#category_name: The name of the top 2 ranking categories
#return only the top 2 movie categories of each customer
(category_rental_counts
 .join(rank_category)
.reset_index()).query('rank_category < 3')

,customer_id,category_name,rental_count,rank_category
2,1,Classics,6,1.0
3,1,Comedy,5,2.0
17,2,Classics,4,2.0
25,2,Sports,5,1.0
27,3,Action,4,1.0
...,...,...,...,...
7732,599,Comedy,3,1.0
7734,599,Family,2,2.0
7735,599,Foreign,2,2.0
7737,599,Horror,3,1.0


In [267]:
total_rent_count = category_rental_counts.groupby(['customer_id'])['rental_count'].sum().to_frame(name='total_rent_count')

In [268]:
avg_rental_count = np.floor(category_rental_counts.groupby(['category_name'])['rental_count'].mean()).to_frame(name='avg_rental_count')

In [271]:
#percentile: How does the customer rank in terms of the top X% compared to all other customers in this film category?
percetile_rank=np.ceil((1-category_rental_counts.groupby(['category_name'])['rental_count'].rank(pct = True))*100).to_frame(name='percentile_rank')

In [272]:
query_category=(category_rental_counts
	.join(rank_category)
 .join(total_rent_count)
 .join(avg_rental_count)
 .join(percetile_rank)
.reset_index())

In [279]:
#average_comparison: How many more films has the customer watched compared to the average DVD Rental Co customer?
query_category['average_comparison'] = np.abs(query_category['rental_count'] - query_category['avg_rental_count'] )

In [286]:
#category_percentage: What proportion of total films watched does this category make up?
query_category['category_percentage'] = np.round(100*query_category['rental_count']/query_category['total_rent_count'],0)

In [287]:
query_category.query('customer_id == 1').sort_values(by='percentile_rank',ascending=True)

,customer_id,category_name,rental_count,average_comparison,rank_category,total_rent_count,avg_rental_count,percentile_rank,category_percentage
2,1,Classics,6,4.0,1.0,32,2.0,1.0,19.0
3,1,Comedy,5,4.0,2.0,32,1.0,2.0,16.0
5,1,Drama,4,2.0,3.0,32,2.0,9.0,12.0
9,1,Music,2,1.0,4.0,32,1.0,38.0,6.0
10,1,New,2,0.0,4.0,32,2.0,43.0,6.0
11,1,Sci-Fi,2,0.0,4.0,32,2.0,47.0,6.0
0,1,Action,2,0.0,4.0,32,2.0,49.0,6.0
12,1,Sports,2,0.0,4.0,32,2.0,52.0,6.0
1,1,Animation,2,0.0,4.0,32,2.0,53.0,6.0
13,1,Travel,1,0.0,5.0,32,1.0,79.0,3.0


In [288]:
query_category.query('rank_category < 3')

,customer_id,category_name,rental_count,average_comparison,rank_category,total_rent_count,avg_rental_count,percentile_rank,category_percentage
2,1,Classics,6,4.0,1.0,32,2.0,1.0,19.0
3,1,Comedy,5,4.0,2.0,32,1.0,2.0,16.0
17,2,Classics,4,2.0,2.0,27,2.0,7.0,15.0
25,2,Sports,5,3.0,1.0,27,2.0,5.0,19.0
27,3,Action,4,2.0,1.0,26,2.0,9.0,15.0
...,...,...,...,...,...,...,...,...,...
7732,599,Comedy,3,2.0,1.0,19,1.0,16.0,16.0
7734,599,Family,2,0.0,2.0,19,2.0,50.0,11.0
7735,599,Foreign,2,0.0,2.0,19,2.0,46.0,11.0
7737,599,Horror,3,2.0,1.0,19,1.0,15.0,16.0
